In [1]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

c:\Users\faust\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Input Problem parameters

Définissons les variables

In [2]:
from extract_parameters import extract_parameters

################################
######## TO DO #################
################################

# choose taille among: "small", "medium", "large"
taille = "small"

In [3]:
################################
######## DO NOT CHANGE #########
################################
conge, qualif, d, p, b, besoin = extract_parameters(taille)

In [4]:
conge

[[1, 1, 0, 1, 1], [1, 0, 1, 1, 1], [1, 1, 1, 1, 1]]

Définissons les paramètres

In [7]:
from define_nums import define_nums

nombre_employes, horizon, nombre_qualif, nombre_projets = define_nums(conge, qualif, d, p, b, besoin)

In [8]:
## u_j, le projet j est réalisé
u = [0]*nombre_projets

## x_i_j_k_n, l'employé k travaille sur le projet j avec la qualif i
## au jour n
x = [[[[0]*horizon for _ in range(nombre_employes)] for _ in range(nombre_projets)] for _ in range(nombre_qualif)]

## begin_j, la date de debut effective du projet j
begin = [0]*nombre_projets

## end_j, la date de fin effective = date de livraison du projet j
end = [0]*nombre_projets